In [1]:
import numpy as np
import os
import shutil
import time
import linecache
from sklearn.metrics import roc_curve, auc,roc_auc_score
import scipy.ndimage as ndi

mode=2# 0: ae 1:cls 2:cls_env
def savefig(gts,scores,fn):
    import matplotlib
    matplotlib.rcParams['backend']='SVG'
    import matplotlib.pyplot as plt
    frame_num=len(scores)
    x = range(frame_num)  #fake data
    y = scores
    fig, ax = plt.subplots()
    
    ax.plot(x, y, 'k')
    ax.grid()
    ax.margins(0) # remove default margins (matplotlib verision 2+)
    dur=[0]
    for i in range(frame_num-1):
        dur.append(gts[i+1]-gts[i])
    dur.append(0-gts[frame_num-1])
    dur=np.array(dur)
    st_f=np.where(dur==1)
    et_f=np.where(dur==-1)
#     print(st_f[0],et_f[0])
    if len(st_f[0])>0:
        for j in range(len(st_f)):
            print(st_f[0][j],et_f[0][j])
            ax.axvspan(st_f[0][j], et_f[0][j], facecolor='red', alpha=0.5)

    
#     plt.show()
    if mode==0:
        plt.savefig('./vis_ae/'+str(fn)+'.svg',format='svg')
    if mode==1:
        plt.savefig('./vis_cls/'+str(fn)+'.svg',format='svg')
    if mode==2:
        plt.savefig('./vis_env/'+str(fn)+'.svg',format='svg')
def score_smoothing2(score,sigma=12):

    r = 10
    if r > score.shape[0] // 2:
        r = score.shape[0] // 2 - 1
    if r % 2 == 0:
        r += 1
    gaussian_temp=np.ones(r*2-1)
    for i in range(r*2-1):
        gaussian_temp[i]=np.exp(-(i-r)**2/(2*sigma**2))/(sigma*np.sqrt(2*np.pi))
    new_score=score
    for i in range(r,score.shape[0]-r):
        new_score[i]=np.dot(score[i-r:i+r-1],gaussian_temp)
    return new_score
def main():
  total=0
  total_eer=0
  # video_frame=[143,121,92,94,100,128,60,3,117,84,47,127,54,50,100,74,42,29,24,27,7]
  video_frame=[1439,1211,923,947,1007,1283,605,36,1175,841,472,1271,549,507,1001,740,426,294,248,273,72]
#   video_frame=[89,75,57,59,62,80,37,2,73,52,29,79,34,31,62,46,26,18,15,17,4]
  sframe=0
  mean_value=np.zeros((21,1))

  num=0
  total_score=np.zeros((15324,1))
  total_gt=np.zeros((15324,1))
  if mode==0:
    reclosspath='total_loss.npy'
    gtpath='total_label.npy'
  if mode==1:
    reclosspath='total_loss_cls.npy'
    gtpath='total_label.npy'
  if mode==2:
    reclosspath='total_loss_evi.npy'
    gtpath='total_label.npy'
  for filename in range(0,21):
    
    count=video_frame[filename]
    
    j = 0;
    loss_total = 0
    eframe=sframe+count;

    gt=np.reshape(np.load(gtpath),(15324,1))
    
    gt=gt[sframe:eframe,:]
    total_gt[sframe:eframe]=gt
    recloss=np.reshape(np.load(reclosspath),(15324,1))
    # precal-minmax
#     maxrec=np.max(recloss)
#     minrec=np.min(recloss)
#     recloss=[(x-minrec)/(maxrec) for x in recloss]
#     recloss=np.reshape(recloss,(15324,1))
    ####
    recloss=recloss[sframe:eframe,:]
    
    gtline=gt
#     reclossline=np.array(recloss)
#     reclossline=np.reshape(reclossline,[count*16,1])
    gtline=np.reshape(gtline,[count*1,1])
    maxrec=np.max(recloss)
    minrec=np.min(recloss)
    score=[x for x in recloss]
    score=[(x-minrec)/(maxrec-minrec) for x in recloss]
    
#     score=ndi.gaussian_filter1d(score, sigma= 10,order=0, output=np.float64, mode='constant',cval=0,truncate=10)
    score=np.reshape(score_smoothing2(np.reshape(np.array(score),(count,))),(count,1))
    total_score[sframe:eframe]=score

#     reclossline=recloss[sframe:eframe,:]
    score=1-np.reshape(score,(count*1,1))
    temp_gt=gt
    gt=gt.reshape((count*1,1))

    sframe=eframe
#     mean_value[filename]=np.mean(reclossline)
    if sum(gt)==count*1 or sum(gt)==0:
      continue;
    
    auc=roc_auc_score(gt,score)
    fpr, tpr, threshold = roc_curve(gt, score, pos_label=1)
    if filename==18:
        np.savetxt('18_gt.txt',gt)
        if mode==0:
            np.savetxt('18_0_scores.txt',score)
        if mode==1:
            np.savetxt('18_1_scores.txt',score)
        if mode==2:
            np.savetxt('18_2_scores.txt',score)
    savefig(gt,score,filename)
    fnr = 1 - tpr
    eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
    EER = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
    total+=auc
    total_eer+=EER
    num+=1
    print (auc,EER,sframe,eframe,np.mean(score),filename,sum(video_frame[:filename]))
  print (total/num,total_eer/num)
  total_score=1-total_score.reshape((15324,1))
  total_gt=total_gt.reshape((15324,1))
  if mode==0:
      np.savetxt("0_Avenue_scroes.txt",total_score)
      np.savetxt("0_Avenue_gt.txt",total_gt)
  if mode==1:
      np.savetxt("1_Avenue_scroes.txt",total_score)
      np.savetxt("1_Avenue_gt.txt",total_gt)
  if mode==2:
      np.savetxt("2_Avenue_scroes.txt",total_score)
      np.savetxt("2_Avenue_gt.txt",total_gt)
  auc=roc_auc_score(total_gt,total_score)
  fpr, tpr, threshold = roc_curve(total_gt, total_score, pos_label=1)
  savefig(total_gt,total_score,22)
  fnr = 1 - tpr
  eer_threshold = threshold[np.nanargmin(np.absolute((fnr - fpr)))]
  EER = fnr[np.nanargmin(np.absolute((fnr - fpr)))]
  total+=auc
  print (auc,EER)
if __name__ == '__main__':
    main()

/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.

77 83
0.890913542914741 0.18137254901960786 1439 1439 0.5974207645160953 0 0
272 320
0.653618137755613 0.5056179775280899 2650 2650 0.8142437462652025 1 1439
294 340
0.7393994390364627 0.3448275862068966 3573 3573 0.7984679428405111 2 2650
379 428
0.9876104857596132 0.043010752688172005 4520 4520 0.7526430110089168 3 3573
468 786
0.844250622997508 0.17924528301886788 5527 5527 0.6443426683350221 4 4520
344 625
0.9000180147721131 0.21678321678321677 6810 6810 0.6113101034781476 5 5527
299 307
0.949718314610532 0.08943089430894313 7415 7415 0.687011784434606 6 6810


/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.

135 183
0.6319955946792474 0.47404844290657444 8626 8626 0.7371038029339442 8 7451
570 583
0.6117624569899326 0.3458646616541353 9467 9467 0.6746644646522906 9 8626
12 34


/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)


...

497 587
0.9790828149774362 0.0888888888888889 13267 13267 0.6714871338428015 14 12266
631 740
0.4535395978423647 0.5963302752293578 14007 14007 0.5742682069343944 15 13267
98 56
0.6337425595238095 0.4765625 14433 14433 0.7244932435250937 16 14007
108 248


/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.

0.9259259259259258 0.15000000000000002 14975 14975 0.7586799148414832 18 14727
64 144
0.6165011459129106 0.48051948051948057 15248 15248 0.6886296721960169 19 14975
13 66
0.955312810327706 0.1132075471698113 15320 15320 0.7213361886604006 20 15248
0.7900760676180737 0.27994561756065284


/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)
/tmp/ipykernel_40910/193146685.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  dur=np.array(dur)


77 83
0.6616219811439117 0.4035560344827587
